# Data Project

From Scrapping to Deploy it in a Streamlit App : Make all the cycle of the Data 



## Summary

- Import libraries
- Extraction of the data With Selenium (+ stockage in mongoDb)
- Treatment of the Data 
- Analyze the Data
- Streamlit Application 

## Import Librarries

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import re
import requests
import csv
import time
import pyarrow.parquet as pq
from pymongo import MongoClient
import numpy as np

## Extraction Data from website

### Website 1   : https://www.wineandco.com/

In [2]:



all_data = []


driver = webdriver.Firefox()


for page in range(0, 3):
    # Navigate to page
    print(f'début scrape de la page {page}')
    driver.get(f'https://www.wineandco.com/vins#page_type=taxon&category_id=2&pageNumber={page}')
    time.sleep(1)

    # Handle cookie dialog
    try:
        cookie_button = driver.find_element(By.CLASS_NAME, 'm-modal__close')
        cookie_button.click()
    except:
        pass

    # Get product links from the page
    product_links = [x.get_attribute('href') for x in driver.find_elements(By.XPATH,'//div[@class="c-wine"]/a')]

    # Loop through product links
    for link in product_links:
        # Navigate to product page
        driver.get(link)
        time.sleep(1)

        # Extract product information
        product = driver.find_element(By.CLASS_NAME, "l-product")

        # Helper function to extract data
        def extract_data(xpath):
            try:
                data = product.find_element(By.XPATH, xpath).get_attribute('textContent')
            except:
                data = ""
            return data

        # Basic product information
        name = extract_data('//h1[@class="l-product__infos__name"]')
        price = extract_data('//span[@class="price-display"]')
        rating = extract_data('//div[@class="l-product__infos__avist"]/span')

        product_info = driver.find_elements(By.CLASS_NAME,"c-product-characteristics__container")

        # Find the div with class 'c-product-characteristics__container'
        # Product characteristics
        product_div = driver.find_element(By.CLASS_NAME, 'c-product-characteristics__container')
        characteristics = []

            # Extract characteristics information
        for ul in product_div.find_elements(By.TAG_NAME, 'ul'):
            for li in ul.find_elements(By.TAG_NAME, 'li'):
                characteristic = {}

                    # Extract label and text
                label = li.find_element(By.CLASS_NAME, 'c-product-characteristics__label').text
                text = li.find_element(By.CLASS_NAME, 'c-product-characteristics__text').text

                    # Store in dictionary
                characteristic[label] = text
                characteristics.append(characteristic)

            # Store data in dictionary
        data = {'name': name, 'price': price, 'rating': rating, 'characteristics': characteristics}
        all_data.append(data)


# Close the webdriver session
driver.quit()


print('fin du scrape')

from pymongo import MongoClient



client = MongoClient('mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+1.8.0')


db = client['Project_final_Ydays']
collection = db['wine']


collection.insert_many(all_data)

print("la donnée est bien arrivée")
client.close()


début scrape de la page 0
début scrape de la page 1
début scrape de la page 2
fin du scrape
la donnée est bien arrivée


## Load The Data From Db

In [3]:


# Connection
client = MongoClient('mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+1.8.0')


db = client['Project_final_Ydays']
collection = db['wine']

# Query , recup sous forme de list
data = list(collection.find())

# conversion en dataframe
df = pd.DataFrame(data)


client.close()

In [4]:
df.head()


,_id,name,price,rating,characteristics
0,644d628e5d3b9d87ff312ef8,\n Domaine la Lôyan...,"9,90 €",,"[{'Domaine': 'Domaine La Lôyane'}, {'Pays': 'F..."
1,644d628e5d3b9d87ff312ef9,\n M. Chapoutier le...,"8,50 €",,"[{'Domaine': 'M. Chapoutier'}, {'Pays': 'Franc..."
2,644d628e5d3b9d87ff312efa,\n La Seigneurie Sa...,"12,00 €",,"[{'Domaine': 'La Seigneurie'}, {'Pays': 'Franc..."
3,644d628e5d3b9d87ff312efb,\n Domaine de Beyss...,"10,90 €",,"[{'Domaine': 'Domaine De Beyssac'}, {'Pays': '..."
4,644d628e5d3b9d87ff312efc,\n Domaine la Lôyan...,"9,90 €",,"[{'Domaine': 'Domaine La Lôyane'}, {'Pays': 'F..."


In [5]:
df.isnull().sum()

_id                0
name               0
price              0
rating             0
characteristics    0
dtype: int64

In [6]:
import pandas as pd
import re






df['name'] = df['name'].str.replace('\n', '')


df['price'] = df['price'].str.replace('€', '')





df = df.astype(str)

def process_characteristics(s):
    # Remove the brackets around the string
    s = s.strip('[]')
    # Split the string by comma, excluding those inside curly braces
    parts = re.findall(r'{[^}]+}|[^,]+', s)
    # Create a dictionary to store the key-value pairs
    d = {}
    # Loop through the parts and extract the key-value pairs
    for part in parts:
        if ':' in part:
           
            key, value = map(str.strip, part.split(':', 1))
           
            d[key] = value
    return d



df['characteristics'] = df['characteristics'].apply(process_characteristics)


df = pd.concat([df.drop(['characteristics'], axis=1), df['characteristics'].apply(pd.Series)], axis=1)


df = df.rename(columns=lambda x: re.sub('\s+', '', x))


print(df.head())

                        _id  \
0  644d628e5d3b9d87ff312ef8   
1  644d628e5d3b9d87ff312ef9   
2  644d628e5d3b9d87ff312efa   
3  644d628e5d3b9d87ff312efb   
4  644d628e5d3b9d87ff312efc   

                                                name   price rating  \
0                              Domaine la Lôyane ...   9,90           
1                              M. Chapoutier les ...   8,50           
2                              La Seigneurie Saum...  12,00           
3                              Domaine de Beyssac...  10,90           
4                              Domaine la Lôyane ...   9,90           

              {'Domaine'    {'Pays'                {'Région' {'Couleur'  \
0   'Domaine La Lôyane'}  'France'}                 'Rhône'}   'Rouge'}   
1       'M. Chapoutier'}  'France'}  'Languedoc-Roussillon'}   'Rouge'}   
2       'La Seigneurie'}  'France'}                 'Loire'}   'Rouge'}   
3  'Domaine De Beyssac'}  'France'}             'Sud-Ouest'}   'Rouge'}   
4   'Domain

In [7]:
df.columns

Index(['_id', 'name', 'price', 'rating', '{'Domaine'', '{'Pays'', '{'Région'',
       '{'Couleur'', '{'Millésime'', '{'Appellation'', '{'Culture'',
       '{'Cépages'', '{"Degréd'alcool"', '{'Températuredeservice'',
       '{'Boireougarder'', '{'Classement''],
      dtype='object')

In [8]:
df

,_id,name,price,rating,{'Domaine',{'Pays',{'Région',{'Couleur',{'Millésime',{'Appellation',{'Culture',{'Cépages',"{""Degréd'alcool""",{'Températuredeservice',{'Boireougarder',{'Classement'
0,644d628e5d3b9d87ff312ef8,Domaine la Lôyane ...,"9,90",,'Domaine La Lôyane'},'France'},'Rhône'},'Rouge'},'2021'},'Côtes du Rhône'},'En conversion biologique'},'Grenache,'14°'},'16°C'},'2020 - 2026'},NaN
1,644d628e5d3b9d87ff312ef9,M. Chapoutier les ...,"8,50",,'M. Chapoutier'},'France'},'Languedoc-Roussillon'},'Rouge'},'2021'},'Côtes du Roussillon Villages'},'Culture raisonnée'},'Syrah,NaN,'16°C - 18°C'},'2022 - 2025'},NaN
2,644d628e5d3b9d87ff312efa,La Seigneurie Saum...,"12,00",,'La Seigneurie'},'France'},'Loire'},'Rouge'},'2020'},'Saumur-Champigny'},'Culture conventionnelle'},'Cabernet Franc'},'14°'},'16-18°C'},'2023 - 2028'},NaN
3,644d628e5d3b9d87ff312efb,Domaine de Beyssac...,"10,90",,'Domaine De Beyssac'},'France'},'Sud-Ouest'},'Rouge'},'2020'},'Côtes-du-Marmandais'},'Culture biologique'},'Merlot,'13.5°'},NaN,NaN,NaN
4,644d628e5d3b9d87ff312efc,Domaine la Lôyane ...,"9,90",,'Domaine La Lôyane'},'France'},'Rhône'},'Blanc'},'2021'},'Côtes du Rhône'},'En conversion biologique'},'Grenache blanc,'13.5°'},'10-12°C'},'2020 - 2024'},NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,644e9c1c5f1576ee8c7d239e,Château Canon 2020...,"180,00",,'Château Canon'},'France'},'Bordeaux'},'Rouge'},'2020'},'Saint-Emilion Grand Cru'},'Culture raisonnée'},'Merlot,'14.5°'},'16-18°C,NaN,'Premier Grand Cru Classé B'}
668,644e9c1c5f1576ee8c7d239f,Marius By Chapouti...,"7,90",,'M. Chapoutier'},'France'},'Languedoc-Roussillon'},'Blanc'},'2020'},"""Pays d'Oc""}",'Culture raisonnée'},'Vermentino'},'13°'},'Autour de 10-12°C'},'2020-2023'},NaN
669,644e9c1c5f1576ee8c7d23a0,François Chidaine ...,"10,00",,'François Chidaine'},'France'},'Loire'},'Blanc'},'2021'},'Touraine'},'Culture conventionnelle'},'Sauvignon'},NaN,'10°C - 12°C'},'2019 - 2021'},NaN
670,644e9c1c5f1576ee8c7d23a1,Les Hauts de Smith...,"15,90",,'Château Smith Haut Lafitte'},'France'},'Bordeaux'},'Rosé'},'2022'},'Bordeaux rosé'},'Culture conventionnelle'},'Cabernet Sauvignon,NaN,'9°C - 10°C'},'2022-2023'},NaN


In [12]:
df_0 = df.filter(like='0', axis=1)

cols_to_drop = list(df_0.columns)



df = df.drop(columns=cols_to_drop)

In [13]:

df.columns = ['id','wine_name', 'price', 'rating', 'domaine', 'pays', 'region', 'couleur', 'millesime','appellation', 'culture', 'cepages', 'degre_alcool(Celsius)', 'temperature_service(Celsius)', 'boire_en', 'garder_jusquà']




df = pd.concat([df, df['domaine'].apply(pd.Series)], axis=1)
df = pd.concat([df, df['pays'].apply(pd.Series)], axis=1)
df = pd.concat([df, df['region'].apply(pd.Series)], axis=1)
df = pd.concat([df, df['couleur'].apply(pd.Series)], axis=1)
df = pd.concat([df, df['millesime'].apply(pd.Series)], axis=1)
df = pd.concat([df, df['appellation'].apply(pd.Series)], axis=1)
df = pd.concat([df, df['culture'].apply(pd.Series)], axis=1)
df = pd.concat([df, df['cepages'].apply(pd.Series)], axis=1)


cols_to_clean =  ['id','wine_name', 'price', 'rating', 'domaine', 'pays', 'region', 'couleur', 'millesime','appellation', 'culture', 'cepages', 'degre_alcool(Celsius)', 'temperature_service(Celsius)', 'boire_en', 'garder_jusquà']
df[cols_to_clean] = df[cols_to_clean].replace({'\'': '', '{': '', '}': '', ' ': '', '°': ''}, regex=True)


df['price'] = df['price'].astype(str).str.replace('\xa0', ' ').str.replace(',', '.').astype(float)


df['degre_alcool(Celsius)'] = df['degre_alcool(Celsius)'].astype(str).str.strip('°').astype(float)




In [17]:

df['millesime'] = df['millesime'].astype(str)




try:
    df[['temperature_min', 'temperature_max']] = df['temperature_service(Celsius)'].str.split('-', expand=True)
    
    df.loc[df['temperature_min'] == 'Frais:autourde10', 'temperature_min'] = '10'
except ValueError:
    df['temperature_min'] = np.nan
    df['temperature_max'] = np.nan


df['temperature_min'] = df['temperature_min'].astype(str).str.strip('C').astype(float)
df['temperature_max'] = df['temperature_max'].astype(str).str.strip('C').astype(float)


df[['boire_en', 'boire_ou2']] = df['boire_en'].str.split('-', expand=True)
df['garder_jusquà'] = df['boire_ou2'].str.strip()


print(df.head())



                         id  \
0  644d628e5d3b9d87ff312ef8   
1  644d628e5d3b9d87ff312ef9   
2  644d628e5d3b9d87ff312efa   
3  644d628e5d3b9d87ff312efb   
4  644d628e5d3b9d87ff312efc   

                                           wine_name  price rating  \
0               DomainelaLôyaneCuvéeBonheurRouge2021    9.9          
1          M.ChapoutierlesVignesdeBila-HautRouge2021    8.5          
2  LaSeigneurieSaumur-ChampignyVieillesVignesàPie...   12.0          
3                   DomainedeBeyssacInitialRouge2020   10.9          
4               DomainelaLôyaneCuvéeBonheurBlanc2021    9.9          

            domaine    pays                region couleur millesime  \
0   DomaineLaLôyane  France                 Rhône   Rouge      2021   
1      M.Chapoutier  France  Languedoc-Roussillon   Rouge      2021   
2      LaSeigneurie  France                 Loire   Rouge      2020   
3  DomaineDeBeyssac  France             Sud-Ouest   Rouge      2020   
4   DomaineLaLôyane  France          

In [18]:
df.columns

Index([                          'id',                    'wine_name',
                              'price',                       'rating',
                            'domaine',                         'pays',
                             'region',                      'couleur',
                          'millesime',                  'appellation',
                            'culture',                      'cepages',
              'degre_alcool(Celsius)', 'temperature_service(Celsius)',
                           'boire_en',                'garder_jusquà',
                                    0,                              0,
                                    0,                              0,
                                    0,                              0,
                                    0,                              0,
                    'temperature_min',              'temperature_max',
                          'boire_ou2'],
      dtype='object')

In [19]:
df = df.drop(columns=[0, 0, 0, 0, 0, 0, 'boire_ou2', 'temperature_service(Celsius)'])


In [21]:
print(df.head())

                         id  \
0  644d628e5d3b9d87ff312ef8   
1  644d628e5d3b9d87ff312ef9   
2  644d628e5d3b9d87ff312efa   
3  644d628e5d3b9d87ff312efb   
4  644d628e5d3b9d87ff312efc   

                                           wine_name  price rating  \
0               DomainelaLôyaneCuvéeBonheurRouge2021    9.9          
1          M.ChapoutierlesVignesdeBila-HautRouge2021    8.5          
2  LaSeigneurieSaumur-ChampignyVieillesVignesàPie...   12.0          
3                   DomainedeBeyssacInitialRouge2020   10.9          
4               DomainelaLôyaneCuvéeBonheurBlanc2021    9.9          

            domaine    pays                region couleur millesime  \
0   DomaineLaLôyane  France                 Rhône   Rouge      2021   
1      M.Chapoutier  France  Languedoc-Roussillon   Rouge      2021   
2      LaSeigneurie  France                 Loire   Rouge      2020   
3  DomaineDeBeyssac  France             Sud-Ouest   Rouge      2020   
4   DomaineLaLôyane  France          